In [1]:
from data_keeper import get_data_keeper

In [2]:
from testing import test_models_with_drugs

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV

In [4]:
get_data_keeper().get_possible_drugs()

[u'ETHA: Ethambutol ',
 u'ISON: Isoniazid ',
 u'RIFM: rifampicin ',
 u'RIFP: Rifapentine ',
 u'PYRA: Pyrazinamide ',
 u'STRE: Streptomycin ',
 u'CYCL: Cycloserine ',
 u'ETHI: Ethionamide/ Prothionamide ',
 u'PARA: Para-aminosalicyclic acid ',
 u'AMIK: Amikacin',
 u' Kanamycin ',
 u'CAPR: Capreomycin ',
 u'OFLO: Ofloxacin ']

In [5]:
import joblib

In [6]:
from wrappers import GridSearchCVWrapper
from wrappers import XGBoostClassifierFeatureImportances as XGB
from wrappers import MatrixCleaningWrapper
from wrappers import SparseWrapper
from wrappers import ModelFeatureSelectionWrapper
from wrappers import ModelBasedFeatureImportanceGetter

In [7]:
from frn import FeatureRelevanceNetworkWrapper

In [8]:
def get_complete_linear_model():
    inner_model = LogisticRegressionCV(Cs=[10 ** i for i in xrange(-4, 4)], solver='liblinear')
    outer_model = GridSearchCV(inner_model, {'penalty': ['l1', 'l2']})
    return MatrixCleaningWrapper(SparseWrapper(outer_model))

In [9]:
def get_complete_tree_based_model():
    cv_params = {'inner_model__inner_model__n_estimators': [1],#, 5, 10, 20, 50, 100], 
                 'feature_selection_threshold_coef': [0.1]}#, 1, 3, 10, 30, 100, 300]}
    return MatrixCleaningWrapper(FeatureRelevanceNetworkWrapper(XGB(n_estimators=100), ModelBasedFeatureImportanceGetter(XGB())))

In [10]:
model = get_complete_tree_based_model()
test_models_with_drugs([('xgboost', model)], [u'ETHI: Ethionamide/ Prothionamide '], metrics=['accuracy', 'confusion matrix'])

cleaner (122, 2718)
{'f949': 1, 'f904': 1, 'f532': 9, 'f606': 10, 'f2604': 9, 'f2193': 15, 'f160': 1, 'f2607': 5, 'f1285': 1, 'f554': 2, 'f1714': 1, 'f800': 1, 'f2391': 3, 'f511': 2, 'f2056': 5, 'f2054': 7, 'f213': 1, 'f1060': 11, 'f117': 6, 'f2375': 3, 'f6': 1, 'f202': 9, 'f225': 1, 'f2100': 1, 'f900': 4, 'f652': 1, 'f2401': 1, 'f2186': 6, 'f1111': 1, 'f972': 3, 'f2177': 1, 'f970': 1, 'f458': 18, 'f2409': 8, 'f996': 30, 'f588': 3, 'f2461': 3, 'f2077': 6, 'f964': 4, 'f932': 5, 'f2469': 1, 'f2379': 1, 'f2586': 8, 'f719': 10, 'f952': 4, 'f2460': 6, 'f955': 17, 'f92': 1, 'f1917': 2, 'f2168': 16, 'f544': 1, 'f353': 3, 'f2380': 11, 'f2381': 12, 'f2040': 1, 'f1873': 3, 'f2161': 8, 'f770': 6, 'f397': 2, 'f798': 1, 'f504': 3, 'f1401': 2, 'f2363': 11, 'f316': 6, 'f211': 3, 'f210': 2, 'f1286': 1, 'f1158': 1, 'f203': 4, 'f1994': 1, 'f1884': 1, 'f2439': 15, 'f2248': 1, 'f35': 9, 'f1543': 2, 'f2356': 8, 'f2418': 8, 'f2374': 2, 'f2373': 3, 'f1937': 3, 'f2370': 8}
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.

{('xgboost',
  u'ETHI: Ethionamide/ Prothionamide '): {'accuracy': 0.82352941176470584, 'confusion matrix': array([[94,  6],
         [18, 18]])}}

In [11]:
X = get_data_keeper().get_common_x()

In [12]:
from testing import results_differ_p_value

In [13]:
results_differ_p_value([0] * 100 + [1] * 32, [0] * 100 + [1] * 32, [0] * 99 + [1] * 32 + [0])

0.077071286548999973

In [ ]:
from generate_subsets_for_common_x import get_ready_generator

In [ ]:
gen = get_ready_generator()

matrix shape before: (142, 13603)
matrix shape after: (142, 3175)


In [ ]:
def extend_with_index(X):
    X = X.copy()
    X['indexes'] = np.arange(X.shape[0])
    return X

In [ ]:
class ExtenderStrategy(object):
    def __init__(self, max_features, max_index=150):
        self._max_features = max_features
        self._generator = None
        self._max_features_sets_storing = 1000000
        assert self._max_features_sets_storing > 2 * max_features
        self._best_feature_sets = list()
        self._max_index = max_index
        
    def set_generator(self, generator):
        self._generator = generator
        
    def _generate_candidates(self):
        for i in xrange(self._generator.get_sets_count()):
            yield self._generator.get_set(i)
            
    def _pre_filter(self, candidate):
        raise NotImplemented()
        
    def _estimate_parameters(self, candidates_generator):
        raise NotImplemented()
        
    def _set_indexes(self, indexes):
        raise NotImplemented()
        
    def _get_candidate_data(self, all_data, candidate):
        raise NotImplemented()
        
    def _get_simple_feature_indexes(self, simple_features, candidate_feature_rows, candidate):
        raise NotImplemented()
        
    def _set_result_feature_sets(self, feature_sets):
        self._result_feature_sets = feature_sets()
        
    def _get_top_priority_feature_sets(self):
        return sorted(self._best_feature_sets, reverse=True)[:self._max_features]
    
    def _clean_storing_feature_sets(self):
        self._best_feature_sets = self._get_top_priority_feature_sets()
    
    def fit(self, simple_features, y, indexes):
        if self._generator is None:
            raise KeyError("generator should be setted")
        self._estimate_parameters(self._generate_candidates())
        self._set_indexes(indexes)
        for candidate in self._generate_candidates():
            if self._pre_filter(candidate):
                candidate_feature_rows = self._get_candidate_feature_rows(simple_features, indexes, candidate)
                y_values = self._get_candidate_y_values(y, indexes, candidate)
                simple_feature_indexes = self._get_simple_feature_indexes(simple_features, candidate_feature_rows, candidate)
                priority = self._get_candidate_priority(candidate, y_values, simple_feature_indexes)
                self._best_feature_sets.append((priority, simple_feature_indexes))
                if len(self._best_feature_sets) > self._max_features_sets_storing:
                    self._clean_storing_feature_sets()
        self._set_result_feature_sets([el[1] for el in self._get_top_priority_feature_sets()])
        return self
    
    def transform(self, simple_features):
        

In [ ]:
class ComplexFeaturesAdderWrapper(BaseEstimator):
    def __init__(self, inner_model, generator, matrix_before_generating, extender_stratedy):
        self.inner_model = inner_model
        self.generator = generator
        self.matrix_before_generating = matrix_before_generating
        self.extender_strategy = extender_strategy
        
    def _get_simple_features(self, indexes):
        return self.matrix_before_generating.as_matrix()[indexes]
    
    def _fit_feature_extender(self, simple_features, y, indexes):
        self.extender_strategy.set_generator(self.generator)
        self.extender_strategy.fit(simple_features, y, indexes)
    
    def _extend_features(self, simple_features):
        return self.extender_strategy.transform(simple_features)
        
    def fit(self, indexes=None, y=None, simple_features=None):
        if y is None:
            raise KeyError("y should be setted")
        if simple_features is None:
            simple_features = self._get_simple_features(indexes)
        else:
            simple_features = simple_features[list(self.matrix_before_generating.columns.values)]
        self._fit_feature_extender(simple_features, y, indexes)
        extended_features = self._extend_features(simple_features)
        self.inner_model.fit(X, y)
        return self
    
    def predict(self, indexes=None, simple_features=None):
        if simple_features is None:
            if indexes is None:
                raise KeyError("either simple_features or indexes should be setted")
            simple_features = self._get_simple_features(indexes)
        extended_features = self._extend_features(simple_features)
        return self.inner_model.predict(extended_features)